In [1]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS

from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from langchain.prompts import PromptTemplate

from langchain.chains import RetrievalQA



In [2]:
## Read the ppdfs from the folder
loader=PyPDFDirectoryLoader("./us_census")

documents=loader.load()

text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)

final_documents=text_splitter.split_documents(documents)
final_documents[0]

Document(page_content='Health Insurance Coverage Status and Type \nby Geography: 2021 and 2022\nAmerican Community Survey Briefs\nACSBR-015Issued September 2023Douglas Conway and Breauna Branch\nINTRODUCTION\nDemographic shifts as well as economic and govern-\nment policy changes can affect people’s access to health coverage. For example, between 2021 and 2022, the labor market continued to improve, which may have affected private coverage in the United States \nduring that time.\n1 Public policy changes included \nthe renewal of the Public Health Emergency, which \nallowed Medicaid enrollees to remain covered under the Continuous Enrollment Provision.\n2 The American \nRescue Plan (ARP) enhanced Marketplace premium subsidies for those with incomes above 400 percent of the poverty level as well as for unemployed people.\n3', metadata={'source': 'us_census/acsbr-015.pdf', 'page': 0})

In [3]:
len(final_documents)

316

In [4]:
## Embedding Using Huggingface
huggingface_embeddings=HuggingFaceBgeEmbeddings(
    model_name="BAAI/bge-small-en-v1.5",      #sentence-transformers/all-MiniLM-l6-v2
    model_kwargs={'device':'cpu'},
    encode_kwargs={'normalize_embeddings':True}

)

/Users/josephmata/Documents/Updated-Langchain/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
import  numpy as np
print(np.array(huggingface_embeddings.embed_query(final_documents[0].page_content)))
print(np.array(huggingface_embeddings.embed_query(final_documents[0].page_content)).shape)

[-8.46568644e-02 -1.19098993e-02 -3.37892659e-02  2.94559151e-02
  5.19159846e-02  5.73839583e-02 -4.10017818e-02  2.74268240e-02
 -1.05128199e-01 -1.58056188e-02  7.94858932e-02  5.64318858e-02
 -1.31765315e-02 -3.41544189e-02  5.81602892e-03  4.72547784e-02
 -1.30747007e-02  3.12993000e-03 -3.44226025e-02  3.08406446e-02
 -4.09086347e-02  3.52737829e-02 -2.43761651e-02 -4.35831770e-02
  2.41503585e-02  1.31986784e-02 -4.84450860e-03  1.92347933e-02
 -5.43912761e-02 -1.42735049e-01  5.15530072e-03  2.93115526e-02
 -5.60810864e-02 -8.53536930e-03  3.14141288e-02  2.76736729e-02
 -2.06188187e-02  8.24231282e-02  4.15425301e-02  5.79655133e-02
 -3.71587127e-02  6.26160763e-03 -2.41389982e-02 -5.61794965e-03
 -2.51715295e-02  5.04971156e-03 -2.52800919e-02 -2.91945087e-03
 -8.24047532e-03 -5.69604412e-02  2.30822805e-02 -5.54220332e-03
  5.11555672e-02  6.09937571e-02  6.49766326e-02 -5.38514331e-02
  2.19109729e-02 -2.54194327e-02 -4.49223109e-02  4.22458947e-02
  4.75252159e-02  7.23251

In [5]:
## VectorStore Creation
vectorstore=FAISS.from_documents(final_documents[:120],huggingface_embeddings)

In [6]:
## Query using Similarity Search
query="WHAT IS HEALTH INSURANCE COVERAGE?"
relevant_docments=vectorstore.similarity_search(query)

print(relevant_docments[0].page_content)

2 U.S. Census Bureau
WHAT IS HEALTH INSURANCE COVERAGE?
This brief presents state-level estimates of health insurance coverage 
using data from the American Community Survey (ACS). The  
U.S. Census Bureau conducts the ACS throughout the year; the 
survey asks respondents to report their coverage at the time of 
interview. The resulting measure of health insurance coverage, 
therefore, reflects an annual average of current comprehensive 
health insurance coverage status.* This uninsured rate measures a 
different concept than the measure based on the Current Population 
Survey Annual Social and Economic Supplement (CPS ASEC). 
For reporting purposes, the ACS broadly classifies health insurance 
coverage as private insurance or public insurance. The ACS defines 
private health insurance as a plan provided through an employer 
or a union, coverage purchased directly by an individual from an 
insurance company or through an exchange (such as healthcare.


In [23]:
retriever=vectorstore.as_retriever(search_type="similarity",search_kwargs={"k":3})
print(retriever)

tags=['FAISS', 'HuggingFaceBgeEmbeddings'] vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x7fba583592d0> search_kwargs={'k': 3}


In [22]:
import os
os.environ['HUGGINGFACEHUB_API_TOKEN']=os.getenv('HUGGINGFACEHUB_API_TOKEN')



The Hugging Face Hub is an platform with over 350k models, 75k datasets, and 150k demo apps (Spaces), all open source and publicly available, in an online platform where people can easily collaborate and build ML together.

In [25]:
from langchain_community.llms import HuggingFaceHub

#accept conditions on huggingface https://huggingface.co/mistralai/Mistral-7B-v0.1.
hf=HuggingFaceHub(
    #repo_id="mistralai/Mistral-7B-v0.1",
    repo_id="TinyLlama/TinyLlama-1.1B-Chat-v0.6",

    model_kwargs={"temperature":0.1,"max_length":500}

)
query="What is the health insurance coverage?"
hf.invoke(query)

'What is the health insurance coverage?\n\nThe health insurance coverage is not mentioned in the text.'

In [26]:
x= hf.invoke(query)

In [27]:
os.environ['HUGGINGFACEHUB_API_TOKEN']=os.getenv('HUGGINGFACEHUB_API_TOKEN')
hf=HuggingFaceHub(
    repo_id="TinyLlama/TinyLlama-1.1B-Chat-v0.6",
    model_kwargs={"temperature":0.1,"max_length":500}
)
query="What is the health insurance coverage?"

'What is the health insurance coverage?\n\nThe health insurance coverage is not mentioned in the text.'

In [40]:
from langchain_community.llms import HuggingFacePipeline


In [20]:
#Hugging Face models can be run locally through the HuggingFacePipeline class.
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline

hf = HuggingFacePipeline.from_model_id(
    model_id="mistralai/Mistral-7B-v0.1",
    #model_id="gpt2",
    task="text-generation",
    pipeline_kwargs={"temperature": .1, "max_new_tokens": 300}
)

llm = hf 
llm.invoke(query)

  warnings.warn(
/Users/josephmata/Documents/Updated-Langchain/venv/lib/python3.10/site-packages/huggingface_hub/file_download.py:1003: UserWarning: Not enough free disk space to download the file. The expected file size is: 9942.98 MB. The target location /Users/josephmata/.cache/huggingface/hub/models--mistralai--Mistral-7B-v0.1/blobs only has 6017.41 MB free disk space.
  warnings.warn(


OSError: [Errno 28] No space left on device

In [28]:
query="""DIFFERENCES IN THE
UNINSURED RATE BY STATE
IN 2022"""

In [29]:
# Call the QA chain with our query.
result = retrievalQA.invoke({"query": query})
print(result['result'])


Use the following piece of context to answer the question asked.
Please try to provide the answer only based on the context

comparison of ACS and CPS ASEC measures 
of health insurance coverage, refer to < www.
census.gov/topics/health/health-insurance/
guidance.html >.
9 Respondents may have more than one 
health insurance coverage type at the time 
of interview. As a result, adding the total 
number of people with private coverage and 
the total number with public coverage will 
sum to more than the total number with any 
coverage.• From 2021 to 2022, nine states 
reported increases in private 
coverage, while seven reported 
decreases (Appendix Table B-2). 
DIFFERENCES IN THE 
UNINSURED RATE BY STATE 
IN 2022
In 2022, uninsured rates at the 
time of interview ranged across 
states from a low of 2.4 percent 
in Massachusetts to a high of 16.6 
percent in Texas, compared to the 
national rate of 8.0 percent.10 Ten 
of the 15 states with uninsured 
10 The uninsured rates in the Distr